# Hands-on: From Model Training to Model Steering

Not long ago, building an AI system meant collecting data, training models, and tuning features — every improvement was an engineering marathon.
Today, with foundation models (FMs), the game has changed. You don’t retrain — you steer.
Small tweaks in instruction, context, or model choice can completely transform output quality.

In this exercise, you’ll see that shift in action.
Using real IMDb movie reviews, you’ll first build a classical sentiment classifier, then experience how a foundation model app can achieve the same goal — but faster, more flexibly, and without a single gradient update.

**Dataset:** [IMDb Movie Reviews (Hugging Face)](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)

### Key Idea
- In **classical ML**, we improve by changing **data**, **features**, or **model training**.
- In **FM apps**, we improve by tuning **instruction**, **context**, or **model choice** — no retraining needed!

## Step 1 — Classical ML Pipeline

We'll train a simple logistic regression classifier on IMDb reviews.

### Levers: data, features, model
- Better **data** → cleaner, more balanced labels
- Smarter **features** → richer representation (e.g., n-grams, TF-IDF)
- Stronger **model** → more accurate predictions

Before we start we need to use the HF Token:
1. Go to Hugging Face
 → Access Tokens.
2. Click New token, choose Read scope (for datasets) → Copy it.
3. Add to the secrets in google colab (HF_TOKEN = **** )

In [ ]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [ ]:
from datasets import load_dataset

# Load full training dataset
full_dataset = load_dataset("imdb", split="train")

# Shuffle and select 5 examples for display
subset = full_dataset.shuffle(seed=42).select(range(5))

# Print examples
for i, record in enumerate(subset):
    print(f"--- Review {i+1} ---")
    print("Text:", record['text'][:200], "…")  # first 200 chars
    print("Label:", "Positive" if record['label'] == 1 else "Negative")
    print()


In [ ]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from collections import Counter

# Load full dataset
full_dataset = load_dataset("imdb", split="train")

# Shuffle and take 1000 samples
dataset = full_dataset.shuffle(seed=42).select(range(1000))

# Extract text and labels
texts = list(dataset['text'])
labels = list(dataset['label'])

# Check distribution
print("Label distribution:", Counter(labels))

# Split train/test with stratification
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

# Vectorize
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train Logistic Regression
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)

# Evaluate
y_pred = clf.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))


In [ ]:
# Example new reviews
new_reviews = [
    "I really enjoyed this movie. The plot was engaging and the actors did a fantastic job!",
    "What a terrible film. It was boring and the story made no sense.",
    "An average movie. Some good parts, but overall not very memorable."
]

# Transform using the same vectorizer
new_reviews_tfidf = vectorizer.transform(new_reviews)

# Predict
predictions = clf.predict(new_reviews_tfidf)

# Display results
for review, pred in zip(new_reviews, predictions):
    print("Review:", review)
    print("Predicted sentiment:", "Positive" if pred == 1 else "Negative")
    print()


### Reflection
- The model learns sentiment *from data patterns*.
- To improve: gather more reviews, add better features, or use a neural model.

But what if we don’t train anything — and simply *ask* a foundation model?

## Step 2 — Foundation Model App (Updated for Hugging Face Inference Client)

In this section, we initialize the **Hugging Face Inference Client**. This allows us to access powerful open-source models hosted on the Hugging Face Hub.

### 🔐 How to Generate your Hugging Face Access Token
To use the Inference API or access gated models (like Gemma), you need an Access Token. Follow these steps derived from the official guide:

1.  **Sign Up:** Go to [huggingface.co/join](https://huggingface.co/join) and create a new account.
2.  **Verify Email (Crucial):** Check your email inbox for a confirmation link.
    * *Note: The "Create new token" button will be **disabled** until you verify your email address.*
3.  **Navigate:** Click on your profile picture in the top-right corner → Select **Access Tokens**.
4.  **Create:** Click the **Create new token** button.
5.  **Configure:**
    * **Name:** Give it a memorable name (e.g., `HF_Token`).
    * **Permissions:** Select `Write` (often required for API usage) or ensure it has default permissions.
6.  **Save:** Click **Create token**. Copy the key immediately and store it securely—you won't be able to see it again later!

> **Security Tip:** Never share your token publicly. If you suspect it has been leaked, go back to the settings page and delete/regenerate it immediately.


### Levers: instruction, context, model
- **Instruction:** phrasing of the prompt
- **Context:** adding background or role info
- **Model:** switching between smaller or larger models

In [ ]:
from google.colab import userdata
from huggingface_hub import InferenceClient

# ------------------------------------------------------------------
# SECURE TOKEN HANDLING
# ------------------------------------------------------------------
# Instructions to set your secret:
# 1. Click the Key icon (🔑) on the left sidebar of Colab.
# 2. Click "Add new secret".
# 3. Name: HF_TOKEN  |  Value: <Paste your Hugging Face token>
# 4. Toggle "Notebook access" to ON.
# ------------------------------------------------------------------

try:
    HF_TOKEN = userdata.get('HF_Token')
except Exception:
    # Fallback: Prompts you to paste the token if the secret isn't set
    import getpass
    print("Secret 'HF_TOKEN' not found. Please paste it below:")
    HF_TOKEN = getpass.getpass("HF Token: ")

client = InferenceClient(token=HF_TOKEN)

# ---- Inference-ready chat models ----
small_chat_model = "google/gemma-2-2b-it"              # supported small chat model
large_chat_model = "zai-org/GLM-4.5"                  # supported larger chat model

context = "You are a film critic analyzing IMDb movie reviews for tone and sentiment."

reviews = [
    "I really enjoyed this movie. The plot was engaging and the actors did a fantastic job!",
    "What a terrible film. It was boring and the story made no sense.",
    "An average movie. Some good parts, but overall not very memorable."
]

def generate_chat(model_id, prompt):
    """
    Uses Hugging Face chat completion via InferenceClient.
    """
    response = client.chat.completions.create(
        model=model_id,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=60
    )
    # Return the assistant message
    return response.choices[0].message["content"]

for i, review in enumerate(reviews, 1):
    print(f"--- Review {i} ---")
    print("Text:", review, "\n")

    # Lever 1 — Instruction only
    p1 = f"Classify the sentiment (Positive or Negative): {review}"
    print("1. Instruction Only →", generate_chat(small_chat_model, p1).strip())

    # Lever 2 — Add Context
    p2 = f"{context}\nClassify the sentiment (Positive or Negative): {review}"
    print("2. +Context →", generate_chat(small_chat_model, p2).strip())

    # Lever 3 — Larger model
    print("3. +Larger Model →", generate_chat(large_chat_model, p2).strip(), "\n")

### 💬 Reflection
- **Instruction:** changes clarity and bias.
- **Context:** adds consistency and reasoning.
- **Model:** increases depth and nuance.

No retraining needed — we *steer* the model instead of *rebuilding* it!

## Step 3 — Summary Table

| Aspect | Classical ML | Foundation Model (FM) |
|--------|---------------|------------------------|
| **Input** | Structured or labeled data | Natural language prompts |
| **Output** | Predictions or scores | Textual or reasoning-based responses |
| **Key Levers** | Data, Features, Model | Instruction, Context, Model |
| **Improvement Cycle** | Retrain with new data | Refine prompts or switch model |

###  Takeaway
- **Classical ML** = learn patterns from data  
- **FM Apps** = steer pre-trained intelligence via prompt design